In [10]:
 !pip install nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [19]:
# !pip install -U sentence-transformers

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
from keras.models import Sequential
from keras.layers import Input, Embedding,TimeDistributed, LSTM, Dropout, Bidirectional, Dense 
import random
from nltk.corpus import wordnet 

from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model, to_categorical
import nltk
nltk.download('punkt')  
from nltk.corpus import wordnet 
from tqdm import tqdm
import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


True

In [21]:
# def cosine(u, v):
#     return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [22]:
# from sentence_transformers import SentenceTransformer


# embidding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



In [23]:
# train_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_train.json", lines=True)
# train_data = train_data.sample(frac=0.01, random_state=42)
# src=train_data["train.SRC"].to_numpy()
# top_decoupled=train_data["train.TOP-DECOUPLED"].to_numpy()


In [24]:
# import logging
# logging.getLogger('sentence_transformers').setLevel(logging.ERROR)
# src_emd =[]
# count=0
# new_src=np.array([])
# new_top_decoupled=np.array([])
# for src_sent,top_sent in tqdm(zip(src,top_decoupled)):
#     flag=False
#     sent_embedding = embidding_model.encode(src_sent)
#     for emd in src_emd:
#         cos=cosine(emd,sent_embedding)
#         # print(cos)
#         if(cos>0.9):
#             flag=True
#             break
#     if(not flag):
#         count+=1
#         src_emd.append(sent_embedding)
#         new_src=np.append(new_src,src_sent)
#         new_top_decoupled=np.append(new_top_decoupled,top_sent)

In [25]:
# file_path = "src.txt"

# # Save to file
# with open(file_path, "w") as file:
#     for line in new_src:
#         file.write(line + "\n")

# file_path = "top_decoupled.txt"

# # Save to file
# with open(file_path, "w") as file:
#     for line in new_top_decoupled:
#         file.write(line + "\n")

In [26]:
# print("count =",count)

In [12]:
file_path = "/kaggle/input/dataset/src.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    new_src = np.array([line.strip() for line in file])

file_path = "/kaggle/input/dataset/top_decoupled.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    new_top_decoupled = np.array([line.strip() for line in file])

In [13]:
def preprosses(src,top_decoupled):
    word_tag=[]
    for src,trg in  zip(src,top_decoupled):
        out=extract_bio_tags(src,trg)
        word_tag.append(out)
    return word_tag




In [14]:
def get_pizza_order_synonyms(word):
    """
    Get synonyms related to the 'pizza order' category.
    """
    # Keywords related to pizza ordering
    pizza_keywords = [
        "pizza", "cheese", "crust", "pepperoni", "mushroom", 
        "topping", "delivery", "order","drink","meat","chicken","size","pizza style","negation"
    ]
    
    synonyms = set()
    for syn in wordnet.synsets(word):
        # Check if the synset definition matches pizza-related keywords
        if any(keyword in syn.definition().lower() for keyword in pizza_keywords):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
                synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
                synonyms.add(synonym)
    
    # Remove the input word from synonyms if it exists
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [15]:
def synonym_replacement(sentence,top_decoupled, num_replacements=1):
    words = sentence.split()  # Tokenize sentence
    words_to_replace = [word for word in words if get_pizza_order_synonyms(word)]  # Only consider words with synonyms

    # Shuffle the list to randomly select words for replacement
    random.shuffle(words_to_replace)

    num_replaced = 0
    for word in words_to_replace:
        if num_replaced >= num_replacements:
            break
        
        synonyms = get_pizza_order_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)  # Pick a random synonym
            words = [synonym if w == word else w for w in words]
            num_replaced += 1
            top_decoupled=re.sub(word,synonym,top_decoupled)

    return " ".join(words),top_decoupled

In [16]:
src=new_src
top_decoupled=new_top_decoupled
print(src.shape)
src_augment=src
top_decoupled_augment=top_decoupled
i=0
for src_itr,top_itr in tqdm(zip(src_augment,top_decoupled_augment)):
    srcc,topp=synonym_replacement(src_itr,top_itr,5)
    src=np.append(src,srcc)
    top_decoupled=np.append(top_decoupled,topp)
   
    

print(src.shape)

    
    
    


(7852,)


7852it [00:38, 203.84it/s]

(15704,)


In [58]:
file_path = "src_65.txt"

# Save to file
with open(file_path, "w") as file:
    for line in src:
        file.write(line + "\n")

file_path = "top_decoupled_65.txt"

# Save to file
with open(file_path, "w") as file:
    for line in top_decoupled:
        file.write(line + "\n")

In [53]:
def extract_bio_tags(src,top_decoupled):
    # Parse the TOP-DECOUPLED structure
    matches = re.findall(r'\(NOT \([^()]+\) \)|\((?:[^()]+)\)|\(NOT \(COMPLEX_TOPPING \((?:.+?) \) \) \)', top_decoupled)
    ind=-1
    if random.random() < 0.5:
         ind=random.randint(0, len(src.split())-1)
    
    # Map words to BIO tags
    bio_tag_map = {}
    for i,match in enumerate(matches):
            
            match=re.sub(r'\(|\)','',match)
            match=match.strip()
            words= match.split()  # Split multi-word values
           
            if(words[0]=="NOT"):
               if("COMPLEX_TOPPING" in words):
                   words.remove("COMPLEX_TOPPING")
                   words.remove("NOT")
                   indx=words.index("TOPPING")
                   quantity=words[1:indx]
                   topping=words[indx+1:]
                   for i, word in enumerate(quantity):
                        if i == 0:
                            bio_tag_map[word]= "B-QUANTITY"  # Begin tag
                        else:
                            bio_tag_map[word]= "I-QUANTITY"  # Inside tag
                   for i, word in enumerate(topping):
                        if i == 0:
                            bio_tag_map[word]= "B-NOT-TOPPING"  # Begin tag
                        else:
                            bio_tag_map[word]= "I-NOT-TOPPING"  # Inside tag
                   continue
               else:
                   tag=words[0]+"-"+words[1]
                   words=words[2:]
            else:
                tag=words[0]
                words=words[1:]
         
            for i, word in enumerate(words):
                if i == 0:
                    bio_tag_map[word]= f"B-{tag}"  # Begin tag
                else:
                    bio_tag_map[word]= f"I-{tag}"  # Inside tag
       
                
            
  
    src_words = src.split()
    word_tag_pairs = [(word, bio_tag_map.get(word, "O")) for word in src_words]
    if ind!=-1:
        word_tag_pairs[ind]=("<UNK>",word_tag_pairs[ind][1])
    
    return word_tag_pairs

 # word_bio_tag_pairs = extract_bio_tags(top_decoupled, src
# out=extract_bio_tags("i want a pizza with sausage bacon and no extra cheese","(ORDER i want (PIZZAORDER (NUMBER a ) pizza with (TOPPING sausage ) (TOPPING bacon ) and no (NOT (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) ) )")
# print(out)





[('i', 'O'), ('want', 'O'), ('<UNK>', 'B-NUMBER'), ('pizza', 'O'), ('with', 'O'), ('sausage', 'B-TOPPING'), ('bacon', 'B-TOPPING'), ('and', 'O'), ('no', 'O'), ('extra', 'B-QUANTITY'), ('cheese', 'B-NOT-TOPPING')]


In [19]:
sentences =preprosses(src,top_decoupled)

In [20]:


maxlen = max([len(s) for s in sentences])
words = [w[0] for sentence in sentences for w in sentence]
words = list(set(words))
words.append("<pad>")
n_words = len(words)
print('Number of unique words:', n_words)


Number of unique words: 346


In [21]:
# print(words)

In [22]:


tags = [w[1] for sentence in sentences for w in sentence]
tags=list(set(tags))
# tags.append("<pad>")
print(tags)
n_tags = len(tags)
print('Number of unique Tags:', n_tags)

['B-CONTAINERTYPE', 'B-DRINKTYPE', 'I-VOLUME', 'I-NOT-STYLE', 'B-SIZE', 'I-STYLE', 'I-TOPPING', 'I-QUANTITY', 'I-DRINKTYPE', 'I-SIZE', 'I-NUMBER', 'B-NOT-TOPPING', 'I-NOT-TOPPING', 'B-TOPPING', 'B-NOT-STYLE', 'B-NUMBER', 'O', 'I-CONTAINERTYPE', 'B-STYLE', 'B-QUANTITY', 'B-VOLUME']
Number of unique Tags: 21


In [23]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
print(tag2idx)

{'B-CONTAINERTYPE': 0, 'B-DRINKTYPE': 1, 'I-VOLUME': 2, 'I-NOT-STYLE': 3, 'B-SIZE': 4, 'I-STYLE': 5, 'I-TOPPING': 6, 'I-QUANTITY': 7, 'I-DRINKTYPE': 8, 'I-SIZE': 9, 'I-NUMBER': 10, 'B-NOT-TOPPING': 11, 'I-NOT-TOPPING': 12, 'B-TOPPING': 13, 'B-NOT-STYLE': 14, 'B-NUMBER': 15, 'O': 16, 'I-CONTAINERTYPE': 17, 'B-STYLE': 18, 'B-QUANTITY': 19, 'B-VOLUME': 20}


In [24]:
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = sequence.pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=n_words - 1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = sequence.pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag2idx['O'])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])
print('X shape', X.shape, 'y shape', y.shape)

X shape (15704, 22) y shape (15704, 22, 21)


In [25]:
class config():
    VOCAB = n_words
    MAX_LEN = maxlen
    N_OUPUT = n_tags
    
    
    EMBEDDING_VECTOR_LENGTH = 256
    N_LSTM_CELLS = 150
    RECURRENT_DROPOUT = 0.4
    
    OUTPUT_ACTIVATION = 'softmax'
    
    LOSS = 'categorical_crossentropy'
    OPTIMIZER = 'adam'
    METRICS = ['accuracy']
    
    MAX_EPOCHS = 5
    

In [26]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
filepath = "model.keras"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
rlp = ReduceLROnPlateau(monitor='loss', patience=3, verbose=1)

model = Sequential()
model.add(
    Embedding(
        input_dim=config.VOCAB, output_dim=config.EMBEDDING_VECTOR_LENGTH, input_length=config.MAX_LEN
    )
)
model.add(
    Bidirectional(
        LSTM(config.N_LSTM_CELLS, return_sequences=True, recurrent_dropout=config.RECURRENT_DROPOUT)
    )
)
model.add(
    TimeDistributed(
        Dense(config.N_OUPUT, activation=config.OUTPUT_ACTIVATION)
    )
)
model.compile(loss=config.LOSS, optimizer=config.OPTIMIZER, metrics=config.METRICS)



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
history = model.fit(x=X, y=y, validation_split=0.1,
    callbacks=[es, ckpt, rlp], epochs=config.MAX_EPOCHS
)

Epoch 1/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.8412 - loss: 0.6448
Epoch 1: loss improved from inf to 0.26193, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.8414 - loss: 0.6439 - val_accuracy: 0.9917 - val_loss: 0.0373 - learning_rate: 0.0010
Epoch 2/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9920 - loss: 0.0334
Epoch 2: loss improved from 0.26193 to 0.03024, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.9920 - loss: 0.0334 - val_accuracy: 0.9944 - val_loss: 0.0239 - learning_rate: 0.0010
Epoch 3/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9949 - loss: 0.0209
Epoch 3: loss improved from 0.03024 to 0.02034, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 20s 46ms/step - accuracy: 0.9949 - loss: 0.0209 - val_accuracy: 0.9947 - val_loss: 0.0186 - learning_rate: 0.0010
Epoch 4/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9955 - loss: 0.0171
Epo

In [28]:
dev_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_dev.json", lines=True)
src_dev=dev_data["dev.SRC"].to_numpy()
top_dev=dev_data["dev.TOP"].to_numpy()


In [54]:
def evaluate(test_src,test_top):
    true_len=0

    true=0
    for src,top in zip(test_src,test_top):
        
        src_words_orginal = src.split() 
        if(len(src_words_orginal)>config.MAX_LEN):
            continue
        true_len+=1
        for word in src_words_orginal:
            if word not in words:
                str=f"(?<=\\s){word}(?=\\s)"
                src=re.sub(str,"<UNK>",src)
        src_words=src.split()
        src_indecies = [word2idx.get(w,word2idx.get("UNK", -1)) for w in src_words]
        pad_token = word2idx.get("<pad>")  
        src_indecies = src_indecies + [pad_token] * (config.MAX_LEN - len(src_words))
        src_indecies = np.expand_dims(src_indecies, axis=0)  # Shape: (1, 25)
        p = model.predict(src_indecies)  
        p = np.argmax(p, axis=-1)  # Get the predicted tag indices
        
        idx2tag = {v: k for k, v in tag2idx.items()}  
        # Iterate through predictions (p) and print the tags
        predicted=[]
        
        for i,w in enumerate(p[0]):  # p is likely a batch, so take the first sequence
            tag_begin=idx2tag[w]
            if tag_begin[0:2]=="I-" and i>0 and (predicted[-1][0:2]!="B-" or tag_begin[2:-1]!=predicted[-1][2:-1]) :
                tag_begin=re.sub("I-","B-",tag_begin)
            predicted.append(tag_begin) 
        matches = re.findall(r'\(NOT \([^()]+\) \)|\((?:[^()]+)\)|\(NOT \(COMPLEX_TOPPING \((?:.+?) \) \) \)', top)
        gold_output=[]
        bio_tag_map={}
        for match in matches:
            match=re.sub(r'\(|\)','',match)
            match=match.strip()
            uu= match.split()  # Split multi-word values
            if(uu[0]=="NOT"):
               if("COMPLEX_TOPPING" in uu):
                   uu.remove("COMPLEX_TOPPING")
                   uu.remove("NOT")
                   indx=uu.index("TOPPING")
                   quantity=uu[1:indx]
                   topping=uu[indx+1:]
                   for i, word in enumerate(quantity):
                        if i == 0:
                            bio_tag_map[word]= "B-QUANTITY"  # Begin tag
                        else:
                            bio_tag_map[word]= "I-QUANTITY"  # Inside tag
                   for i, word in enumerate(topping):
                        if i == 0:
                            bio_tag_map[word]= "B-NOT-TOPPING"  # Begin tag
                        else:
                            bio_tag_map[word]= "I-NOT-TOPPING"  # Inside tag
                   continue
               else:
                   gold_output.append(uu[0]+"-"+uu[1])
                   uu=uu[2:]
            else:
                gold_output.append(uu[0])
                uu=uu[1:]
            for i, word in enumerate(uu):
                if i == 0:
                    bio_tag_map[word]= f"B-{gold_output[-1]}"  # Begin tag
                else:
                    bio_tag_map[word]= f"I-{gold_output[-1]}"  # Inside tag
        word_tag_pairs = [(word, bio_tag_map.get(word, "O")) for word in src_words_orginal]
        word_tag_pairs=[wor[1] for wor in word_tag_pairs ]
        word_tag_pairs = word_tag_pairs + ["O"] * (config.MAX_LEN - len(word_tag_pairs))

        acc=[a == b for a, b in zip(word_tag_pairs, predicted)]
        if(all(acc)==True):
            true+=1
        else:
            print(src,word_tag_pairs,predicted)
        
    print("accuracy",true/true_len*100,"%")

In [55]:
evaluate(src_dev,top_dev)

i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage (ORDER i want to order (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING sausage ) and (TOPPING black olives ) ) and (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING pepperoni ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) and (PIZZAORDER (NUMBER three ) (SIZE large ) pizzas with (TOPPING pepperoni ) and (TOPPING sausage ) ) )
five medium pizzas with tomatoes and ham (ORDER (PIZZAORDER (NUMBER five ) (SIZE medium ) pizzas with (TOPPING tomatoes ) and (TOPPING ham ) ) )
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
i need to order one large vegetarian pizza with extra banana peppers (ORDER i need to order (PIZZAORDER (NUMBER one ) (SIZE large ) (STYLE vegetarian ) pizza with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING banana peppers ) ) ) )
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
i'd like to

In [45]:
def predict(sentence):
    
  
        src_words_orginal = sentence.split() 
        if(len(src_words_orginal)>config.MAX_LEN):
            return 
        for word in src_words_orginal:
            if word not in words:
                str=f"(?<=\\s){word}(?=\\s)"
                sentence=re.sub(str,"<UNK>",sentence)
        src_words=sentence.split()
        src_indecies = [word2idx.get(w,word2idx.get("UNK", -1)) for w in src_words]
        pad_token = word2idx.get("<pad>")  
        src_indecies = src_indecies + [pad_token] * (config.MAX_LEN - len(src_words))
        src_indecies = np.expand_dims(src_indecies, axis=0)  # Shape: (1, 25)
        p = model.predict(src_indecies)  
        p = np.argmax(p, axis=-1)  # Get the predicted tag indices
        
        idx2tag = {v: k for k, v in tag2idx.items()}  
        # Iterate through predictions (`p`) and print the tags
        predicted=[]
        
        for i,w in enumerate(p[0]):  # `p` is likely a batch, so take the first sequence
            predicted.append(idx2tag[w]) 
        print(sentence,predicted)
       
       
        
            

predict("I will take 2 pizza large with foul and salami")    
                
       
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
I <UNK> <UNK> 2 pizza large with <UNK> and salami ['O', 'O', 'O', 'B-NUMBER', 'O', 'O', 'O', 'B-TOPPING', 'O', 'B-TOPPING', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [56]:
model.save("model_65.keras")